In [124]:
import cx_Oracle
import os
import pandas as pd
#!pip insall cx_Oracle

In [125]:
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']

In [126]:
import datetime 

In [127]:
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

ProgrammingError: Oracle Client library has already been initialized

In [434]:
connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

In [7]:
def get_df_from_db(table_name='all_tables'):
    """
    table_name 에는 해당하는 table_name만 넣으면 자동으로 pandas 데이터 프레임으로 변환해준다.
    """
    rows_or = []
    try:
        for row_data in cursor.execute("select * from " + table_name):
            rows_or.append(list(row_data))
        colum_num = len(list(row_data))
    except: 
        print("Error -> 테이블명이 정확하지 않습니다.")
        raise
    num = int(colum_num)
    col_name = []
    for i in range(num):
        col_name.append('column_'+str(i))

    df = pd.DataFrame(columns=col_name)
    for idx,row in enumerate(rows_or):
        df.loc[idx] = row
    if table_name == 'all_tables' :
        df = df[df['column_0'] == 'DM_HOPITAL'].rename(columns={'column_1':'조회가능한_테이블명'})[['조회가능한_테이블명']]. \
            reset_index().drop(columns='index')
        return df
    else :    
        return df

In [190]:
get_df_from_db('SHIPMENT_INFORMATION')

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17
0,138,STO0147988,105325285,114845190,1,2020-12-21,2116655,None,7104,바이나리,2021-01-04,2021-01-05,None,None,대리점,UPD3039393,None,None
1,139,STO0147988,105325285,114845190,1,2020-12-21,2116656,50415,4664,바이나리,2021-01-04,2021-01-05,None,None,대리점,UPD3039388,None,None
2,140,STO0147988,105325285,114845190,1,2020-12-21,2116656,None,38646,바이나리,2021-01-04,2021-01-05,None,None,대리점,UPD3039385,None,None
3,141,STO0147988,105325285,114845190,1,2020-12-21,2116656,None,7104,바이나리,2021-01-04,2021-01-05,None,None,대리점,UPD3039387,None,None
4,142,STO0147988,105325278,114836014,1,2020-12-21,2116409,1390,1390,바이오토프,2021-01-04,2021-01-05,2021-01-05,None,대리점,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20482,20405,STO0165498,115389193,140031675,1,2022-10-19,IR-SM00516072,1784,None,KR_LSW,2022-10-27,None,None,None,None,UPD3349889,None,None
20483,20406,STO0165498,115389194,140033160,1,2022-10-19,IR-SM00516818,1521,None,KR_KHK,2022-10-27,None,None,None,None,UPD3349891,None,None
20484,20407,SISA22036945,115384447,140001728,1,2022-10-18,800652,1953,None,엠프로테크,2022-10-27,None,None,None,HKK_Mprotech(Arkgen)_Q-290100V1_pure25M2,SGD000204235,None,None
20485,20408,4551883674,115388143,140021395,1,2022-10-21,650062990,241,None,FTC,2022-10-27,None,None,None,WR-017298_Demo con_GWL,LGM000655267,None,None


In [287]:
# Shipment_information

ws_si = wb_cy.sheets[2]
last_row = ws_si.range("A1048576").end('up').row
last_col = ws_si.range("XFD9").end('left').column
col_name = ws_si.range((9,1),(9,last_col))
data_si = ws_si.range((9,1),(last_row,last_col)).options(pd.DataFrame, index=False, numbers=int,dates =datetime.date).value

data_si

,SI_INDEX,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM,ORDER_TOTAL,UNIT_PRICE,SHIP_TO,ARRIVAL_DATE,SHIP_DATE,POD_DATE,FOR_FREE,REMARK,PARCELS_NO,COMMENT,STATUS
0,1,7764059026,None,None,None,None,None,None,None,DHL특송건,2021-01-04,2021-01-06,None,무상,None,None,None,None
1,2,9531213646,None,None,None,None,None,None,None,DHL특송건,2021-01-04,2021-01-08,None,None,None,None,None,None
2,3,437779237520,105354422,115291677,1,None,2116872,206,206,바이나리,2021-01-18,2021-01-18,2021-01-20,None,대리점,PYD002357006,None,None
3,4,STO0148367,105359564,115374865,None,None,2116946,1766,1766,이효바이오사이언스,2021-01-19,2021-01-19,2021-01-20,None,대리점,UPD3047151,None,None
4,5,STO0148367,105359564,115374865,None,None,2116947,166,166,이효바이오사이언스,2021-01-19,2021-01-19,2021-01-20,None,대리점,UPD3047151,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20482,20483,STO0165597,115394010,140138477,1,2022-10-24,2125298,2017,2017,바이나리,2022-10-31,None,None,None,대리점,UPD3351722,None,None
20483,20484,STO0165597,115394011,140137540,1,2022-10-24,800665,8727,None,비엔씨코리아,2022-10-31,None,None,None,SHL_Microzyme_Q-298606 V0_AKTApure150M,UPD3351766,None,None
20484,20485,STO0165597,115394011,140149337,1,2022-10-24,800665,None,None,비엔씨코리아,2022-10-31,None,None,None,SHL_Microzyme_Q-298606 V0_AKTApure150M,UPD3352123,None,None
20485,20486,STO0165597,115394012,140139530,1,2022-10-24,IR-SM00515698,2699,None,KR_CCK,2022-10-31,None,None,None,None,UPD3351878,None,None


In [288]:
# 컬럼이름
col_name.value

# 컬럼개수
len(col_name.value)

18

In [289]:
value_list = []
for i in range(1,len(col_name.value)+1):
    value_list.append(":"+str(i))
value_list[10:]

[':11', ':12', ':13', ':14', ':15', ':16', ':17', ':18']

In [290]:
val_str = "':1', ':2', ':3', ':4', ':5', ':6', ':7', ':8', ':9', ':10', ':11', ':12', ':13', ':14', ':15', ':16', ':17', ':18'"
val_str.replace("'","")

':1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16, :17, :18'

In [291]:
data_si.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20487 entries, 0 to 20486
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   SI_INDEX       20487 non-null  int64 
 1   AWB_NO         20487 non-null  object
 2   TRIP_NO        20182 non-null  object
 3   SHIPMENT_NM    20176 non-null  object
 4   NM_OF_PACKAGE  18545 non-null  object
 5   INVOICE_DATE   20316 non-null  object
 6   ORDER_NM       20369 non-null  object
 7   ORDER_TOTAL    14639 non-null  object
 8   UNIT_PRICE     7109 non-null   object
 9   SHIP_TO        20460 non-null  object
 10  ARRIVAL_DATE   20103 non-null  object
 11  SHIP_DATE      19284 non-null  object
 12  POD_DATE       14993 non-null  object
 13  FOR_FREE       65 non-null     object
 14  REMARK         17748 non-null  object
 15  PARCELS_NO     20191 non-null  object
 16  COMMENT        2610 non-null   object
 17  STATUS         0 non-null      object
dtypes: int64(1), object(17)
me

In [296]:
str_list=list(data_si.columns)


In [297]:
data_si= pd.concat([data_si[str_list[0]],data_si[str_list[1:]].astype(str)],axis=1)
data_si

,SI_INDEX,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM,ORDER_TOTAL,UNIT_PRICE,SHIP_TO,ARRIVAL_DATE,SHIP_DATE,POD_DATE,FOR_FREE,REMARK,PARCELS_NO,COMMENT,STATUS
0,1,7764059026,None,None,None,None,None,None,None,DHL특송건,2021-01-04,2021-01-06,None,무상,None,None,None,None
1,2,9531213646,None,None,None,None,None,None,None,DHL특송건,2021-01-04,2021-01-08,None,None,None,None,None,None
2,3,437779237520,105354422,115291677,1,None,2116872,206,206,바이나리,2021-01-18,2021-01-18,2021-01-20,None,대리점,PYD002357006,None,None
3,4,STO0148367,105359564,115374865,None,None,2116946,1766,1766,이효바이오사이언스,2021-01-19,2021-01-19,2021-01-20,None,대리점,UPD3047151,None,None
4,5,STO0148367,105359564,115374865,None,None,2116947,166,166,이효바이오사이언스,2021-01-19,2021-01-19,2021-01-20,None,대리점,UPD3047151,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20482,20483,STO0165597,115394010,140138477,1,2022-10-24,2125298,2017,2017,바이나리,2022-10-31,None,None,None,대리점,UPD3351722,None,None
20483,20484,STO0165597,115394011,140137540,1,2022-10-24,800665,8727,None,비엔씨코리아,2022-10-31,None,None,None,SHL_Microzyme_Q-298606 V0_AKTApure150M,UPD3351766,None,None
20484,20485,STO0165597,115394011,140149337,1,2022-10-24,800665,None,None,비엔씨코리아,2022-10-31,None,None,None,SHL_Microzyme_Q-298606 V0_AKTApure150M,UPD3352123,None,None
20485,20486,STO0165597,115394012,140139530,1,2022-10-24,IR-SM00515698,2699,None,KR_CCK,2022-10-31,None,None,None,None,UPD3351878,None,None


In [298]:
data_si.fillna('None')

,SI_INDEX,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM,ORDER_TOTAL,UNIT_PRICE,SHIP_TO,ARRIVAL_DATE,SHIP_DATE,POD_DATE,FOR_FREE,REMARK,PARCELS_NO,COMMENT,STATUS
0,1,7764059026,None,None,None,None,None,None,None,DHL특송건,2021-01-04,2021-01-06,None,무상,None,None,None,None
1,2,9531213646,None,None,None,None,None,None,None,DHL특송건,2021-01-04,2021-01-08,None,None,None,None,None,None
2,3,437779237520,105354422,115291677,1,None,2116872,206,206,바이나리,2021-01-18,2021-01-18,2021-01-20,None,대리점,PYD002357006,None,None
3,4,STO0148367,105359564,115374865,None,None,2116946,1766,1766,이효바이오사이언스,2021-01-19,2021-01-19,2021-01-20,None,대리점,UPD3047151,None,None
4,5,STO0148367,105359564,115374865,None,None,2116947,166,166,이효바이오사이언스,2021-01-19,2021-01-19,2021-01-20,None,대리점,UPD3047151,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20482,20483,STO0165597,115394010,140138477,1,2022-10-24,2125298,2017,2017,바이나리,2022-10-31,None,None,None,대리점,UPD3351722,None,None
20483,20484,STO0165597,115394011,140137540,1,2022-10-24,800665,8727,None,비엔씨코리아,2022-10-31,None,None,None,SHL_Microzyme_Q-298606 V0_AKTApure150M,UPD3351766,None,None
20484,20485,STO0165597,115394011,140149337,1,2022-10-24,800665,None,None,비엔씨코리아,2022-10-31,None,None,None,SHL_Microzyme_Q-298606 V0_AKTApure150M,UPD3352123,None,None
20485,20486,STO0165597,115394012,140139530,1,2022-10-24,IR-SM00515698,2699,None,KR_CCK,2022-10-31,None,None,None,None,UPD3351878,None,None


In [299]:
df_list = data_si.fillna("None").values.tolist()
df_tuple =list(pd.Series(df_list).map(tuple))

In [300]:
df_tuple[0]

(1,
 '7764059026',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'DHL특송건',
 '2021-01-04',
 '2021-01-06',
 'None',
 '무상',
 'None',
 'None',
 'None',
 'None')

In [128]:
exm_tu = tuple(df_list[0])

In [185]:
table = 'SHIPMENT_INFORMATION'
values = ':1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16, :17, :18'.replace(' ','')

In [186]:
my_query = f"INSERT INTO {table} VALUES({values})"
my_query

'INSERT INTO SHIPMENT_INFORMATION VALUES(:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18)'

In [187]:
cursor.executemany

<function Cursor.executemany>

In [301]:
cursor.executemany(my_query,df_tuple,batcherrors= True)

In [302]:
cursor.execute("commit")

In [303]:
col_names = list(data_si.columns)

In [312]:
def get_df_from_db(query="select * from SHIPMENT_INFORMATION"):
    
    df_si = pd.DataFrame(cursor.execute(query).fetchall(),columns=col_names).sort_values(str_list[0])
    df_si.index = df_si[col_names[0]]
    df_si =df_si.drop(columns=col_names[0])
    
    return df_si
df_si = get_df_from_db()

In [313]:
df_si

,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM,ORDER_TOTAL,UNIT_PRICE,SHIP_TO,ARRIVAL_DATE,SHIP_DATE,POD_DATE,FOR_FREE,REMARK,PARCELS_NO,COMMENT,STATUS
SI_INDEX,,,,,,,,,,,,,,,,,
1,7764059026,None,None,None,None,None,None,None,DHL특송건,2021-01-04,2021-01-06,None,무상,None,None,None,None
2,9531213646,None,None,None,None,None,None,None,DHL특송건,2021-01-04,2021-01-08,None,None,None,None,None,None
3,437779237520,105354422,115291677,1,None,2116872,206,206,바이나리,2021-01-18,2021-01-18,2021-01-20,None,대리점,PYD002357006,None,None
4,STO0148367,105359564,115374865,None,None,2116946,1766,1766,이효바이오사이언스,2021-01-19,2021-01-19,2021-01-20,None,대리점,UPD3047151,None,None
5,STO0148367,105359564,115374865,None,None,2116947,166,166,이효바이오사이언스,2021-01-19,2021-01-19,2021-01-20,None,대리점,UPD3047151,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20483,STO0165597,115394010,140138477,1,2022-10-24,2125298,2017,2017,바이나리,2022-10-31,None,None,None,대리점,UPD3351722,None,None
20484,STO0165597,115394011,140137540,1,2022-10-24,800665,8727,None,비엔씨코리아,2022-10-31,None,None,None,SHL_Microzyme_Q-298606 V0_AKTApure150M,UPD3351766,None,None
20485,STO0165597,115394011,140149337,1,2022-10-24,800665,None,None,비엔씨코리아,2022-10-31,None,None,None,SHL_Microzyme_Q-298606 V0_AKTApure150M,UPD3352123,None,None


In [436]:
sht_row_idx = "si_1042A1762C1773A1926C1931A6271C6273A6703A6948C6964A7233C7234A7236A8118A8127"

sht_row_idx = 'si_19807C19811'
sht_row_idx = sht_row_idx[3:].split("A")

# for idx,nms in enumerate(sht_row_idx):
#     if "C" in nms :
#         sht_row_idx[idx] = sht_row_idx[idx].split("C",)
#         in_list_1 = int(sht_row_idx[idx][0])
#         in_list_2 = int(sht_row_idx[idx][1])
#         diff = int(in_list_2)- int(in_list_1)
#         cons_list = []
#         for i in range(diff+1):
#             i = in_list_1 + i
#             cons_list.append(i)
#         sht_row_idx[idx] = cons_list
#     else:
#         sht_row_idx[idx] = [int(sht_row_idx[idx])]



        
# si_idx_list = sum(sht_row_idx, [])
# pd.Series(si_idx_list)

In [424]:
si_idx_list
si_idx_list_mapped = list(map(lambda e: e-1,si_idx_list))
si_idx_list_mapped

[1041,
 1761,
 1762,
 1763,
 1764,
 1765,
 1766,
 1767,
 1768,
 1769,
 1770,
 1771,
 1772,
 1925,
 1926,
 1927,
 1928,
 1929,
 1930,
 6270,
 6271,
 6272,
 6702,
 6947,
 6948,
 6949,
 6950,
 6951,
 6952,
 6953,
 6954,
 6955,
 6956,
 6957,
 6958,
 6959,
 6960,
 6961,
 6962,
 6963,
 7232,
 7233,
 7235,
 8117,
 8126]

In [425]:
df_si.iloc[pd.Series(si_idx_list_mapped)]

,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM,ORDER_TOTAL,UNIT_PRICE,SHIP_TO,ARRIVAL_DATE,SHIP_DATE,POD_DATE,FOR_FREE,REMARK,PARCELS_NO,COMMENT,STATUS
SI_INDEX,,,,,,,,,,,,,,,,,
1042,EWR0305070,105382943,115649437,1,2021-01-27,2114101,18223,None,SK바이오사이언스,2021-02-01,2021-05-13,None,None,KJJ_SK_PO7700009546_BAGS,PYD002371857,None,None
1762,EWR0305025,105421546,116170985,1,2021-02-24,2114101,63444,None,SK바이오사이언스,2021-03-11,2021-05-13,None,None,KJJ_SK_PO7700009546_BAGS,PYD002399179,None,None
1763,EWR0305025,105421546,116169974,1,2021-02-24,2114101,None,None,SK바이오사이언스,2021-03-11,2021-05-13,None,None,KJJ_SK_PO7700009546_BAGS,PYD002399177,None,None
1764,EWR0305025,105421546,116269306,1,2021-02-24,2114101,None,None,SK바이오사이언스,2021-03-11,2021-05-13,None,None,KJJ_SK_PO7700009546_BAGS,PYD002399165,None,None
1765,EWR0305025,105421546,116269306,1,2021-02-24,2114101,None,None,SK바이오사이언스,2021-03-11,2021-05-13,None,None,KJJ_SK_PO7700009546_BAGS,PYD002399180,None,None
1766,EWR0305025,105421546,116269306,1,2021-02-24,2114101,None,None,SK바이오사이언스,2021-03-11,2021-05-13,None,None,KJJ_SK_PO7700009546_BAGS,PYD002399175,None,None
1767,EWR0305025,105421546,116269306,1,2021-02-24,2114101,None,None,SK바이오사이언스,2021-03-11,2021-05-13,None,None,KJJ_SK_PO7700009546_BAGS,PYD002399089,None,None
1768,EWR0305025,105421546,116269306,1,2021-02-24,2114101,None,None,SK바이오사이언스,2021-03-11,2021-05-13,None,None,KJJ_SK_PO7700009546_BAGS,PYD002399181,None,None
1769,EWR0305025,105421546,116269306,1,2021-02-24,2114101,None,None,SK바이오사이언스,2021-03-11,2021-05-13,None,None,KJJ_SK_PO7700009546_BAGS,PYD002399176,None,None


In [445]:
row_idx =sht_row_idx

base_qry = "select * from SHIPMENT_INFORMATION where SI_INDEX "
col_name = 'SI_INDEX '
or_word = ' or '
btw = "between "
and_word = "and "
for idx,nm in enumerate(row_idx) :
    if len(row_idx) == 1:
            if "C" in nm:
                c_split = row_idx[idx].split("C")
                base_qry = base_qry + btw + c_split[0] + " " + and_word + c_split[1]

            else : 
                base_qry = base_qry + "= " + row_idx[0]
    else :
        if idx == 0:
            if "C" in nm:
                c_split = row_idx[0].split("C")
                base_qry = base_qry + btw + c_split[0] + " " + and_word + c_split[1] + or_word

            else : 
                base_qry = base_qry + "= " + row_idx[0] + or_word

        elif idx == len(row_idx) -1 :
            if "C" in nm:
                c_split = row_idx[idx].split("C")
                base_qry = base_qry + col_name + btw + c_split[0] + " " + and_word + c_split[1]

            else : 
                base_qry = base_qry + col_name + "= " + row_idx[0]
        else :
            if "C" in nm:
                c_split = row_idx[idx].split("C")
                base_qry = base_qry + col_name + btw + c_split[0] + " " + and_word + c_split[1] + or_word

            else : 
                base_qry = base_qry + col_name + "= " + row_idx[0] + or_word
            
base_qry

'select * from SHIPMENT_INFORMATION where SI_INDEX between 19807 and 19811'

In [446]:
qry_data = cursor.execute(base_qry).fetchall()
df_qryed = pd.DataFrame(qry_data,columns=str_list)
df_qryed

,SI_INDEX,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM,ORDER_TOTAL,UNIT_PRICE,SHIP_TO,ARRIVAL_DATE,SHIP_DATE,POD_DATE,FOR_FREE,REMARK,PARCELS_NO,COMMENT,STATUS
0,19807,STO0165276,115368677,139742489,1,2022-10-07,2125043,11937,None,삼성바이오로직스,2022-10-18,None,None,None,SVC_PO4500033119,UPD3344236,None,None
1,19808,STO0165276,115368677,139742489,1,2022-10-07,2125043,None,None,삼성바이오로직스,2022-10-18,None,None,None,SVC_PO4500033119,UPD3344241,None,None
2,19809,STO0165276,115368677,139742489,1,2022-10-07,2125043,None,None,삼성바이오로직스,2022-10-18,None,None,None,SVC_PO4500033119,UPD3344245,None,None
3,19810,STO0165276,115368677,139742489,1,2022-10-07,2125043,None,None,삼성바이오로직스,2022-10-18,None,None,None,SVC_PO4500033119,UPD3344246,None,None
4,19811,STO0165276,115368677,139742489,1,2022-10-07,2125043,None,None,삼성바이오로직스,2022-10-18,None,None,None,SVC_PO4500033119,UPD3344244,None,None


In [308]:


qry_data = cursor.execute("""
        select * from SHIPMENT_INFORMATION 
        where SHIP_DATE in (:name1, :name2)""",
        name1="None", name2="None").fetchall()

In [309]:
df_qryed = pd.DataFrame(qry_data,columns=str_list)

In [459]:
def get_query_df(row_idx):
    row_idx = row_idx[3:].split("A")
    base_qry = "select * from SHIPMENT_INFORMATION where SI_INDEX "
    col_name = 'SI_INDEX '
    or_word = ' or '
    btw = "between "
    and_word = "and "
    for idx,nm in enumerate(row_idx) :
        if len(row_idx) == 1:
                if "C" in nm:
                    c_split = row_idx[idx].split("C")
                    base_qry = base_qry + btw + c_split[0] + " " + and_word + c_split[1]

                else : 
                    base_qry = base_qry + "= " + row_idx[0]
        else :
            if idx == 0:
                if "C" in nm:
                    c_split = row_idx[0].split("C")
                    base_qry = base_qry + btw + c_split[0] + " " + and_word + c_split[1] + or_word

                else : 
                    base_qry = base_qry + "= " + row_idx[0] + or_word

            elif idx == len(row_idx) -1 :
                if "C" in nm:
                    c_split = row_idx[idx].split("C")
                    base_qry = base_qry + col_name + btw + c_split[0] + " " + and_word + c_split[1]

                else : 
                    base_qry = base_qry + col_name + "= " + row_idx[0]
            else :
                if "C" in nm:
                    c_split = row_idx[idx].split("C")
                    base_qry = base_qry + col_name + btw + c_split[0] + " " + and_word + c_split[1] + or_word

                else : 
                    base_qry = base_qry + col_name + "= " + row_idx[0] + or_word
    qry_data = cursor.execute(base_qry).fetchall()
    df_qryed = pd.DataFrame(qry_data,columns=str_list)
    return df_qryed

In [457]:
sht_row_idx = wb_cy.sheets[0].range("D3:D5").value
sht_row_idx

['si_29665',
 'si_1042A1762C1773A1926C1931A6271C6273A6703A6948C6964A7233C7234A7236A8118A8127',
 'si_19807C19811']

In [465]:
df_out = get_query_df(sht_row_idx[2]).sort_values("SI_INDEX")

In [472]:
str(wb_cy.sheets[0].range("F3:F5").value[2].date())

'2023-01-30'

In [473]:
df_out['SHIP_DATE'] = str(wb_cy.sheets[0].range("F3:F5").value[2].date())

In [475]:
df_out

,SI_INDEX,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM,ORDER_TOTAL,UNIT_PRICE,SHIP_TO,ARRIVAL_DATE,SHIP_DATE,POD_DATE,FOR_FREE,REMARK,PARCELS_NO,COMMENT,STATUS
0,19807,STO0165276,115368677,139742489,1,2022-10-07,2125043,11937,None,삼성바이오로직스,2022-10-18,2023-01-30,None,None,SVC_PO4500033119,UPD3344236,None,None
1,19808,STO0165276,115368677,139742489,1,2022-10-07,2125043,None,None,삼성바이오로직스,2022-10-18,2023-01-30,None,None,SVC_PO4500033119,UPD3344241,None,None
2,19809,STO0165276,115368677,139742489,1,2022-10-07,2125043,None,None,삼성바이오로직스,2022-10-18,2023-01-30,None,None,SVC_PO4500033119,UPD3344245,None,None
3,19810,STO0165276,115368677,139742489,1,2022-10-07,2125043,None,None,삼성바이오로직스,2022-10-18,2023-01-30,None,None,SVC_PO4500033119,UPD3344246,None,None
4,19811,STO0165276,115368677,139742489,1,2022-10-07,2125043,None,None,삼성바이오로직스,2022-10-18,2023-01-30,None,None,SVC_PO4500033119,UPD3344244,None,None
